In [ ]:
import pandas as pd

df = pd.read_csv('data/dirty_data.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
contain_nulls = df[
df.SNOW.isnull() | df.SNWD.isna()\
| pd.isnull(df.TOBS) | pd.isna(df.WESF)\
| df.inclement_weather.isna()
]
contain_nulls.shape[0]

In [ ]:
contain_nulls.head(10)


In [ ]:
df[df.inclement_weather == 'NaN'].shape[0]


In [ ]:
import numpy as np
df[df.inclement_weather == np.nan].shape[0]

In [ ]:

df[df.inclement_weather.isna()].shape[0]

In [ ]:
df[df.SNWD.isin([-np.inf, np.inf])].shape[0]

In [ ]:
import numpy as np

def get_inf_count(df):
"""Find the number of inf/-inf values per column in the dataframe"""
return {
col : df[df[col].isin([np.inf, -np.inf])].shape[0] for col in df.columns
}

get_inf_count(df)

In [ ]:
pd.DataFrame({
'np.inf Snow Depth': df[df.SNWD == np.inf].SNOW.describe(),
'-np.inf Snow Depth': df[df.SNWD == -np.inf].SNOW.describe()
}).T

In [ ]:
df.describe(include='object')

In [ ]:
df[df.duplicated()].shape[0]

In [ ]:
df[df.duplicated(keep=False)].shape[0]

In [ ]:

df[df.duplicated(['date', 'station'])].shape[0]

In [ ]:
df[df.duplicated()].head()

In [ ]:
# save this information for later
station_qm_wesf = df[df.station == '?'].WESF

# sort ? to the bottom
df.sort_values('station', ascending=False, inplace=True)

# drop duplicates based on the date column keeping the first occurrence
# which will be the valid station if it has data
df_deduped = df.drop_duplicates('date').drop(
# remove the station column because we are done with it
# and WESF because we need to replace it later
columns=['station', 'WESF']

).sort_values('date').assign( # sort by the date
# add back the WESF column which will be properly matched because of the index
WESF=station_qm_wesf
)

df_deduped.shape

In [ ]:
df_deduped.shape()

In [ ]:
df_deduped.dropna().shape

In [ ]:
df_deduped.dropna(how='all').shape

In [ ]:
df_deduped.dropna(
how='all', subset=['inclement_weather', 'SNOW', 'SNWD']
).shape

In [ ]:
df_deduped.dropna(axis='columns', thresh=df_deduped.shape[0]*.75).columns

In [ ]:
df_deduped.loc[:'WESF'].fillna(0, inplace=True)
df_deduped.head()

In [ ]:
df_deduped.assign(
TMAX=lambda x: x.TMAX.replace(5505, np.nan).fillna(method='ffill'),
TMIN=lambda x: x.TMIN.replace(-40, np.nan).fillna(method='ffill')
).head()

In [ ]:

df_deduped.assign(
SNWD=lambda x: np.nan_to_num(x.SNWD)
).head()

In [ ]:
df_deduped.assign(
  TMAX=lambda x: x.TMAX.replace(5505, np.nan).fillna(x.TMAX.median()),
  TMIN=lambda x: x.TMIN.replace(-40, np.nan).fillna(x.TMIN.median()),
  # average of TMAX and TMIN
  TOBS=lambda x: x.TOBS.fillna((x.TMAX + x.TMIN) / 2)
).head()

In [ ]:
df_deduped.assign(
   # make TMAX and TMIN NaN where appropriate
   TMAX=lambda x: x.TMAX.replace(5505, np.nan),
   TMIN=lambda x: x.TMIN.replace(-40, np.nan)
).set_index('date').apply(
   # rolling calculations will be covered in chapter 4, this is a rolling 7 day median
   # we set min_periods (# of periods required for calculation) to 0 so we always get a result
   lambda x: x.fillna(x.rolling(7, min_periods=0).median())
).head(10)


In [ ]:
df_deduped.assign(
   # make TMAX and TMIN NaN where appropriate
  TMAX=lambda x: x.TMAX.replace(5505, np.nan),
  TMIN=lambda x: x.TMIN.replace(-40, np.nan),
  date=lambda x: pd.to_datetime(x.date)
).set_index('date').reindex(
  pd.date_range('2018-01-01', '2018-12-31', freq='D')
).apply(
  lambda x: x.interpolate()
).head(10)